In [154]:
import neo4j as neo
import json

In [155]:
url = 'bolt://localhost:7687'
auth = ('neo4j', 'neo4j_')

In [2]:
def _run_query(tx, query, **args):
    """
    Run one query statement in transaction, and return result
    """
    result = tx.run(query, **args)
    return result

def _run_transaction(statement_list):
    # Make neo4j driver object
    driver = neo.GraphDatabase.driver(url, auth=auth)
    try:
        # Start neo4j session
        with driver.session() as session:
            results = []
            for statement in statement_list:
                query = statement['statement']
                args = statement.get('args', {})
                method = statement.get('method', 'READ')
                # Put each statement into transaction
                if method == 'WRITE':
                    result = session.write_transaction(_run_query, query, **args)
                else:
                    result = session.read_transaction(_run_query, query, **args)
                
                results.append(result)
                
            return results
    finally:
        driver.close()
        
def pretty_output(results_lists):
    for n in range(0, len(results_lists)):
        results = results_lists[n].values()
        for m in range(0, len(results)):
            for i in range(0, len(results[m])):
                print('{}.{}.{}: {}'.format(n, m, i, results[m][i]))
            print()
        print()

In [157]:
"""
Create or merge object in neo4j
"""

statements = [
    {
        'statement': 'MERGE (person:Person {id: {id}, name: {name}, gender: {gender}}) '
                     'RETURN person',
        'args': {
            'id': 1,
            'name': 'Alvin',
            'gender': 'M' 
        },
        'method': 'WRITE'
    },
    {
        'statement': 'MERGE (person:Person {id: {id}, name: {name}, gender: {gender}}) '
                     'RETURN person',
        'args': {
            'id': 2,
            'name': 'Emma',
            'gender': 'F' 
        },
        'method': 'WRITE'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Node id=2983 labels={'Person'} properties={'gender': 'M', 'name': 'Alvin', 'id': 1}>


1.0.0: <Node id=2984 labels={'Person'} properties={'gender': 'F', 'name': 'Emma', 'id': 2}>




In [158]:
"""
Update exist object
"""

statements = [
    {
        'statement': 'MATCH (person:Person)'
                     'WHERE person.id = {id} '
                     'SET person.birthday = {birthday} '
                     'RETURN person',
        'args': {
            'id': 1,
            'birthday': '1981-03-17'
        },
        'method': 'WRITE'
    },
    {
        'statement': 'MATCH (person:Person)'
                     'WHERE person.id = {id} '
                     'SET person.birthday = {birthday} '
                     'RETURN person',
        'args': {
            'id': 2,
            'birthday': '1985-03-29'
        },
        'method': 'WRITE'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Node id=2983 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>


1.0.0: <Node id=2984 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>




In [159]:
"""
Build relationship
"""

statements = [
    {
        'statement': 'MATCH (p1:Person), (p2:Person)'
                     'WHERE p1.id = {id1} AND p2.id = {id2} '
                     'MERGE (p1)-[r:Married]->(p2) '
                     'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        },
        'method': 'WRITE'
    },
    {
        'statement': 'MATCH (p1:Person), (p2:Person) '
                     'WHERE p1.id = {id1} AND p2.id = {id2} '
                     'MERGE (p2)-[r:Married]->(p1) '
                     'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        },
        'method': 'WRITE'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Node id=2983 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>
0.0.1: <Node id=2984 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>
0.0.2: Married


1.0.0: <Node id=2983 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>
1.0.1: <Node id=2984 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>
1.0.2: Married




In [160]:
"""
Create object and build relationship
"""

statements = [
    {
        'statement': 'MATCH (person:Person) '
                     'WHERE person.id = {person_id} '
                     'MERGE (cat:Cat { id: {cat_id}, name: {cat_name} }) '
                     'MERGE (person)-[r:Feed]->(cat) '
                     'RETURN person, cat, type(r)',
        'args': {
            'person_id': 1,
            'cat_id': 3,
            'cat_name': "Lily"
        },
        'method': 'WRITE'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Node id=2983 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>
0.0.1: <Node id=3567 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>
0.0.2: Feed




In [161]:
"""
Find person who feed cat Lily
"""

statements = [
    {
        'statement': 'MATCH (person:Person)-[r:Feed]->(cat:Cat) '
                     'WHERE cat.name = {cat_name} '
                     'RETURN person, cat, type(r)',
        'args': {
            'cat_name': "Lily"
        }
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Node id=2983 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>
0.0.1: <Node id=3567 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>
0.0.2: Feed




In [162]:
"""
Find persons who has relationship with cat Lily
"""

statements = [
    {
        'statement': 'MATCH (person:Person)-[r*]->(cat:Cat) '
                     'WHERE cat.name = {cat_name} '
                     'RETURN DISTINCT person.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

pretty_output(_run_transaction(statements))

0.0.0: Alvin

0.1.0: Emma




In [163]:
"""
Find who has relationship with cat Lily
"""

statements = [
    {
        'statement': 'MATCH (p1:Person)-[:Married]-(p2:Person)-[r:Feed*]->(c:Cat) '
                     'WHERE c.name={cat_name} '
                     'RETURN DISTINCT p1.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

pretty_output(_run_transaction(statements))

0.0.0: Emma




In [164]:
"""
Find graph to cat Lily
"""

statements = [
    {
        'statement': 'MATCH g = (person:Person)-[r*]->(cat:Cat) '
                     'WHERE cat.name = {cat_name} '
                     'RETURN g, length(g)',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Path start=<Node id=2983 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> end=<Node id=3567 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=1>
0.0.1: 1

0.1.0: <Path start=<Node id=2984 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}> end=<Node id=3567 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=2>
0.1.1: 2

0.2.0: <Path start=<Node id=2983 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> end=<Node id=3567 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=3>
0.2.1: 3




In [165]:
"""
Find last children node in all relationship
"""

statements = [
    {
        'statement': 'MATCH ()-[*]->(n:Cat) '
                     'WHERE NOT (n)-[]->() '
                     'RETURN DISTINCT n'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Node id=3567 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>




In [166]:
"""
Match results
"""

driver = neo.GraphDatabase.driver(url, auth=auth)

try:
    
    def _query_last_node(tx):
        return tx.run('MATCH ()-[r*]->(n:Cat) '
                      'WHERE NOT (n)-[]->() '
                      'RETURN DISTINCT n')

    def _print_node(node):
        print('Node physical id is: {}'.format(node.id))
        print('Node label is: {}'.format(next(iter(node.labels))))
        print('Node property keys are: {}'.format(list(node.keys())))
        print('Node property values are: {}'.format(list(node.values())))
        print('Node property id is: {}'.format(node['id']))
        print('Node property name is: {}'.format(node.get('name')))

    with driver.session() as session:
        result = session.read_transaction(_query_last_node).values() # Get result values
        node = result[0][0]
        _print_node(node)

        
    def _print_relationship(relationships):
        for r in relationships:
            r = r[0][0]
            print('--->')
            print('\tRelationship physical id is: {}'.format(r.id))
            print('\tRelationship label is: {}'.format(r.type))
            print('\tRelationship start node id is [id={}, name={}]'.format(
                r.start_node['id'], r.start_node['name']))
            print('\tRelationship start node id is [id={}, name={}]'.format(
                r.end_node['id'], r.end_node['name']))

    def _query_relationship(tx):
        return tx.run('MATCH ()-[r*]->(n:Cat) '
                      'WHERE NOT (n)-[]->() '
                      'RETURN DISTINCT r, n')

    with driver.session() as session:
        result = session.read_transaction(_query_relationship).values() # Get result values
        _print_relationship(result)
        
finally:
    driver.close()

Node physical id is: 3567
Node label is: Cat
Node property keys are: ['name', 'id']
Node property values are: ['Lily', 3]
Node property id is: 3
Node property name is: Lily
--->
	Relationship physical id is: 7932
	Relationship label is: Feed
	Relationship start node id is [id=None, name=None]
	Relationship start node id is [id=3, name=Lily]
--->
	Relationship physical id is: 7931
	Relationship label is: Married
	Relationship start node id is [id=None, name=None]
	Relationship start node id is [id=None, name=None]
--->
	Relationship physical id is: 8621
	Relationship label is: Married
	Relationship start node id is [id=None, name=None]
	Relationship start node id is [id=None, name=None]


In [167]:
"""
Delete nodes with relationships
"""

statements = [
    {
        'statement': 'MATCH ()-[mr:Married]->() '
                     'MATCH ()-[fr:Feed]->() '
                     'OPTIONAL MATCH (ps:Person) '
                     'OPTIONAL MATCH(cs:Cat) '
                     'DELETE mr, fr, ps, cs '
                     'RETURN mr, fr, ps, cs'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Relationship id=8621 nodes=(<Node id=2983 labels=set() properties={}>, <Node id=2984 labels=set() properties={}>) type='Married' properties={}>
0.0.1: <Relationship id=7932 nodes=(<Node id=2983 labels=set() properties={}>, <Node id=3567 labels=set() properties={}>) type='Feed' properties={}>
0.0.2: <Node id=2983 labels=set() properties={}>
0.0.3: <Node id=3567 labels=set() properties={}>

0.1.0: <Relationship id=8621 nodes=(<Node id=2983 labels=set() properties={}>, <Node id=2984 labels=set() properties={}>) type='Married' properties={}>
0.1.1: <Relationship id=7932 nodes=(<Node id=2983 labels=set() properties={}>, <Node id=3567 labels=set() properties={}>) type='Feed' properties={}>
0.1.2: <Node id=2984 labels=set() properties={}>
0.1.3: <Node id=3567 labels=set() properties={}>

0.2.0: <Relationship id=7931 nodes=(<Node id=2984 labels=set() properties={}>, <Node id=2983 labels=set() properties={}>) type='Married' properties={}>
0.2.1: <Relationship id=7932 nodes=(<Node id=298

In [169]:
"""
Import from csv
"""

statements = [
    {
        'statement': 'LOAD CSV WITH HEADERS FROM "file:///data.csv" AS line '
                     'MERGE (p1:Person {id: line["id"], name: line["name"]}) '
                     'MERGE (p2:Person {id: line["married"]}) '
                     'MERGE (p1)-[:Married]->(p2) '
                     'RETURN p1, p2'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Node id=3568 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>
0.0.1: <Node id=3569 labels={'Person'} properties={'name': 'Emma', 'id': '2'}>

0.1.0: <Node id=3569 labels={'Person'} properties={'name': 'Emma', 'id': '2'}>
0.1.1: <Node id=3568 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>




In [172]:
"""
List all lucence fulltext search analyzers
"""

statements = [
    {
        'statement': 'CALL db.index.fulltext.listAvailableAnalyzers()'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: lithuanian
0.0.1: Lithuanian analyzer with stemming and stop word filtering.

0.1.0: simple
0.1.1: A simple analyzer that tokenizes at non-letter boundaries. No stemming or filtering. Works okay for most European languages, but is terrible for languages where words are not separated by spaces, such as many Asian languages.

0.2.0: latvian
0.2.1: Latvian analyzer with stemming and stop word filtering.

0.3.0: cjk
0.3.1: CJK - Chinese/Japanese/Korean - analyzer. Terms are normalised and case-folded. Produces bi-grams, and filters out stop words.

0.4.0: sorani
0.4.1: Sorani Kurdish analyzer with stemming and stop word filtering.

0.5.0: stop
0.5.1: Stop analyzer tokenizes at non-letter characters, and filters out English stop words. This differs from the 'classic' and 'standard' analyzers in that it makes no effort to recognize special terms, like likely product names, URLs or email addresses.

0.6.0: indonesian
0.6.1: Indonesian analyzer with stemming and stop word filtering.

0.

In [203]:
"""
Create fulltext index
"""

statements = [
    {
        'statement': 'CALL db.index.fulltext.createNodeIndex("ix_name", ["Person", "Cat"], ["name"], {analyzer: "cjk"})'
    }
]

pretty_output(_run_transaction(statements))

In [204]:
"""
Query by fulltext index
"""

statements = [
    {
        'statement': 'CALL db.index.fulltext.queryNodes("ix_name", "Alvin") YIELD node, score '
                     'RETURN node, score'
    }
]

pretty_output(_run_transaction(statements))

0.0.0: <Node id=3568 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>
0.0.1: 1.0




In [200]:
"""
Query by fulltext index
"""

statements = [
    {
        'statement': 'CALL db.index.fulltext.drop("ix_name") '
    }
]

pretty_output(_run_transaction(statements))